In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

tokenizer = GPT2TokenizerFast.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

In [2]:
import pandas as pd
import datasets

patient_notes = pd.read_csv('data/en/patient_notes.csv')
patient_notes.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [3]:
from tools.text_tools import process_en

patient_notes['pn_history'] = patient_notes.apply(lambda x: process_en(x['pn_history']), axis=1)
patient_notes.head()


,pn_num,case_num,pn_history
0,0,0,"<AGE> year old male, has come to the student h..."
1,1,0,<AGE> year old male with recurrent palpitation...
2,2,0,Dillon cleveland is a <AGE> year old male pati...
3,3,0,A <AGE> year old m c/o palpitation started 3 m...
4,4,0,<AGE> year old male with no pmh here for evalu...


In [4]:
csv_file = 'data/en/patient_notes_processed.csv'
patient_notes.to_csv(csv_file, index=False)

In [5]:
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples['pn_history'], truncation=True, max_length=512, padding="max_length")

studmed_ds_train = datasets.load_dataset('csv', data_files=csv_file, split='train[:80%]').shuffle()
studmed_ds_test = datasets.load_dataset('csv', data_files=csv_file, split='train[80%:]').shuffle()

tokenized_datasets_train = studmed_ds_train.map(tokenize_function, batched=True)
tokenized_datasets_test = studmed_ds_test.map(tokenize_function, batched=True)

Using custom data configuration default-09ecaaf82b3d7ae5


100%|██████████| 1/1 [00:00<00:00, 253.54it/s]


Dataset csv downloaded and prepared to /Users/andrei/.cache/huggingface/datasets/csv/default-09ecaaf82b3d7ae5/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


Using custom data configuration default-09ecaaf82b3d7ae5
Reusing dataset csv (/Users/andrei/.cache/huggingface/datasets/csv/default-09ecaaf82b3d7ae5/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
  3%|▎         | 1/34 [00:00<00:04,  6.70ba/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


...

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


 78%|███████▊  | 7/9 [00:01<00:00,  6.35ba/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


100%|██████████| 9/9 [00:01<00:00,  5.31ba/s]

Ignored unknown kwarg option direction


In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()